In [2]:
!pip install transformers

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 7, Finished, Available)

In [3]:
import pandas as pd

# Replace 'path_to_file' with the actual path to your file
df = pd.read_csv('Users/hassan-kamran/mini_sample.xls')

df = df[['reviewText', 'overall']]

print(df)

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 8, Finished, Available)

                                             reviewText  overall
0     This is one my must have books. It is a master...      5.0
1     This book provides a reflection that you can a...      5.0
2     I first read THE PROPHET in college back in th...      5.0
3     A timeless classic.  It is a very demanding an...      5.0
4     Reading this made my mind feel like a still po...      5.0
...                                                 ...      ...
1021  To make my point I'll be as concise as possibl...      1.0
1022  This is a book that I can't put down, until 2 ...      4.0
1023  I started this book with the idea that it woul...      4.0
1024  This novel was beautifully written.  Amy Tan o...      4.0
1025  The Professor and the Madman deals with the ro...      4.0

[1026 rows x 2 columns]


In [4]:
import math
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2Model

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 9, Finished, Available)

2024-03-24 21:10:47.021768: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
# divide into test and train
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], df['overall'],
              shuffle=True, test_size=0.2, random_state=42, stratify=df['overall'])


StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 10, Finished, Available)

In [6]:
MAX_LENGTH = math.ceil((X_train.apply(lambda x: len(str(x).split())).mean()))+2
MAX_LENGTH

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 11, Finished, Available)

112

In [7]:
PAD_TOKEN = "<|pad|>"
EOS_TOKEN = "<|endoftext|>"

# this will download and initialize the pre trained tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2",
    pad_token=PAD_TOKEN,
    eos_token=EOS_TOKEN,
    max_length=MAX_LENGTH,
    is_split_into_words=True)

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 12, Finished, Available)

In [8]:
X_train = [str(ex) + EOS_TOKEN for ex in X_train]
X_test = [str(ex) + EOS_TOKEN for ex in X_test]

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 13, Finished, Available)

In [9]:
X_train_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)['input_ids'] for x in X_train]
X_test_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)['input_ids'] for x in X_test]

X_train_in = tf.squeeze(tf.convert_to_tensor(X_train_), axis=1)
X_test_in = tf.squeeze(tf.convert_to_tensor(X_test_), axis=1)

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 14, Finished, Available)

/home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
2024-03-24 21:10:58.779663: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 21:10:58.780021: I tensorflow/core/common_runtime/process_uti

In [10]:
X_train_in[0]

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 15, Finished, Available)

<tf.Tensor: shape=(112,), dtype=int32, numpy=
array([16190, 10732,   281, 22284, 21559,  3081,  1626,  2405,    13,
         220,  2773,  3925,   423,   407, 45569,   428,  3721,    13,
         220,  2773,  3925,   466,   407,  6537,   326,   257,  2829,
        1487,   460,  2251,   257,  3236, 13389,    13,   220, 10928,
         345,  1265,   281,  1593,  3785,   329,  5608,   878,   673,
         393,   339,  1364,  3240,    30,   220,  5334,  5608,   561,
         307,  4847,   546,  1204,   326,   345,  1541,  2993,    11,
         475,  9775, 16453,    13,  1318,   389,  2829, 33099,   284,
        1061,   326,   460,   787,   345,   257,  1365,  1048,  1626,
         534,   898,  2951,    13,    38,   571,  2596,  3544,   383,
       13583, 25009,   436, 28485,   284, 19128,   268,  7183,   351,
        2829, 10815,   484,   460,   779,   284,  2107,   257,   517,
        5899,   290,  8768,  1204], dtype=int32)>

In [11]:
X_train_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)["attention_mask"] for x in X_train]
X_test_mask_ = [tokenizer(str(x), return_tensors='tf', max_length=MAX_LENGTH, truncation=True, pad_to_max_length=True, add_special_tokens=True)["attention_mask"] for x in X_test]

X_train_mask = tf.squeeze(tf.convert_to_tensor(X_train_mask_), axis=1)
X_test_mask = tf.squeeze(tf.convert_to_tensor(X_test_mask_), axis=1)

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 16, Finished, Available)

In [12]:
model = TFGPT2Model.from_pretrained("gpt2", use_cache=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id)
model.training = True

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 17, Finished, Available)

All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [13]:
model.resize_token_embeddings(len(tokenizer))

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 18, Finished, Available)

In [14]:
for layer in model.layers:
    layer.trainable = False

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 19, Finished, Available)

In [15]:
model.summary()

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 20, Finished, Available)

Model: "tfgpt2_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 124440576 
 r)                                                              
                                                                 
Total params: 124,440,576
Trainable params: 0
Non-trainable params: 124,440,576
_________________________________________________________________


In [16]:
input = tf.keras.layers.Input(shape=(None,), dtype='int32')
mask = tf.keras.layers.Input(shape=(None,), dtype='int32')
x = model(input, attention_mask=mask)
#x = x.last_hidden_state[:, -1]
x = tf.reduce_mean(x.last_hidden_state, axis=1)
x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 21, Finished, Available)

In [17]:
clf = tf.keras.Model([input, mask], output)
     



StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 22, Finished, Available)

In [18]:
clf.summary()

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 23, Finished, Available)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 tfgpt2_model (TFGPT2Model)     TFBaseModelOutputWi  124440576   ['input_1[0][0]',                
                                thPastAndCrossAtten               'input_2[0][0]']                
                                tions(last_hidden_s                                               
                                tate=(None, None, 7                                           

In [19]:
base_learning_rate = 0.0005
optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
#loss=tf.keras.losses.BinaryCrossentropy()
loss=tf.keras.losses.SparseCategoricalCrossentropy()

clf.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 24, Finished, Available)

In [20]:
callbacks = tf.keras.callbacks.EarlyStopping(
        monitor="accuracy", verbose=1, patience=3, restore_best_weights=True)

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 25, Finished, Available)

In [21]:
def map_sentiment(value):
    if value in [1, 2]:
        return 0
    elif value == 3:
        return 1
    elif value in [4, 5]:
        return 2
    else:
        # Handle other cases, if needed
        return None  # or any default value you prefer


StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 26, Finished, Available)

In [22]:
y_train_ = y_train.map(map_sentiment)
y_test_ = y_test.map(map_sentiment)

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 27, Finished, Available)

In [23]:
y_train_in = tf.constant(y_train_, dtype=tf.int32)
y_test_in = tf.constant(y_test_, dtype=tf.int32)


StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 28, Finished, Available)

In [24]:
tf.config.experimental_run_functions_eagerly(True)

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 29, Finished, Available)

From /tmp/ipykernel_4246/2874510872.py:1: experimental_run_functions_eagerly (from tensorflow.python.eager.def_function) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [25]:
history = clf.fit([X_train_in, X_train_mask], y_train_in, epochs=1, batch_size=32, validation_split=0.2, callbacks=callbacks)

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 30, Finished, Available)

/home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


21/21 [==============================] - 379s 18s/step - loss: 9.5037 - accuracy: 0.1875 - val_loss: 1.1376 - val_accuracy: 0.5854


In [26]:
clf.evaluate([X_test_in, X_test_mask], y_test_in)

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 31, Finished, Available)

7/7 [==============================] - 40s 6s/step - loss: 1.0131 - accuracy: 0.6650


[1.013129711151123, 0.6650485396385193]

In [27]:
clf.training = False
y_pred = clf.predict([X_test_in, X_test_mask])

StatementMeta(a30545b6-b56d-480e-96fd-847dfba9cab9, 9, 32, Submitted, Running)

5/7 [====================>.........] - ETA: 12s

In [ ]:
y_pred_out = tf.math.argmax(y_pred, axis=-1)
y_pred_out

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test_in, y_pred_out))